In [1]:
import shutup
import torch
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from tsgcn.simulation import MsprimeSimulation
from tsgcn.util import get_idle_gpu
from tsgcn.data import TreeSequenceData, TreeSequencesDataset
from tsgcn.model import BiGCNModel, BiGCNEncoder

import tskit

shutup.please()

/home/murillor/miniconda3/envs/gcn/lib/python3.10/site-packages/torch_geometric/typing.py:31: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /home/murillor/miniconda3/envs/gcn/lib/python3.10/site-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/home/murillor/miniconda3/envs/gcn/lib/python3.10/site-packages/torch_geometric/typing.py:42: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /home/murillor/miniconda3/envs/gcn/lib/python3.10/site-packages/torch_sparse/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "
/home/murillor/miniconda3/envs/gcn/lib/python3.10/site-packages/stdpopsim/catalog/HomSap/demographic_models.py:158: FutureWarning: Calling int on a single element Series is depr

In [2]:
# this device variable will get used later during training
device = torch.device(f"cuda:{get_idle_gpu()}" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
## CONSTANTS
seed = 11379
num_reps = 200
sp_name = "HomSap"
chrom = "chr13"
model_name ="OutOfAfrica_3G09"
sample_size=10
HomSap_chr13_OOA = MsprimeSimulation(seed, num_reps, sp_name, chrom, model_name, "data/raw/", sample_size, n_threads=50)


In [4]:
# Just exploring the output of convert_tseq
from tsgcn.util import convert_tseq
ts=tskit.load(HomSap_chr13_OOA.ts_paths[0])
eix, ei, nf, sq = convert_tseq(ts)
a=TreeSequenceData(edge_index=eix, edge_interval=ei, x = nf, sequence_length=sq)

In [5]:
# Importing our dataset
num_windows = 200
dataset = TreeSequencesDataset("data/", HomSap_chr13_OOA.sims_path,seeds=HomSap_chr13_OOA.seed_array)
#dataset.process_y(num_windows=num_windows)

In [6]:
dataset[0]

TreeSequenceData(x=[141489, 1], edge_index=[2, 864663], edge_interval=[2, 864663], sequence_length=114364328.0, y=[200])

In [7]:
t = BiGCNEncoder(1,4)
a = t(dataset[0])

In [8]:
dataset.len()

200

In [9]:
from torch.utils.data import random_split


train_set, valid_set, test_set = random_split(dataset,[120, 40, 40])

trainloader = DataLoader(train_set, batch_size=1, shuffle=True)
validloader = DataLoader(valid_set, batch_size=1, shuffle=True)
testloader = DataLoader(test_set, batch_size=1, shuffle=True)

In [10]:
def get_y(batch):
    #return batch.x[:,0].unsqueeze(1)
    return batch.y.unsqueeze(1)

In [13]:
breaks = np.linspace(0, HomSap_chr13_OOA.contig.length, num_windows+1)
model = BiGCNModel(device, num_encoder_in_features=dataset.num_features, breaks=breaks, pooling="windowed_sum")
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.0005)
num_epochs=50
criterion = torch.nn.MSELoss()

for epoch in range(num_epochs):
    epoch_loss = 0
    model.train()
    num_batches = 0
    for batch in trainloader:
        num_batches+=1
        batch = batch.to(device)
        optimizer.zero_grad()
        output = model(batch)
        #print(output.shape, batch.x[:,0].unsqueeze(1).shape)
        loss = criterion(output,get_y(batch))
        loss.backward()
        epoch_loss += loss.item()
        optimizer.step()
    
    train_avg_loss = epoch_loss / num_batches
    val_loss = 0
    model.eval()
    num_batches = 0
    for batch in validloader:
        num_batches+=1
        batch.to(device)
        output = model(batch)
        loss = criterion(output,get_y(batch))
        val_loss += loss.item()
    val_avg_loss = val_loss/num_batches
    
    
    print(f"Epochs: {epoch} | epoch avg. loss: {train_avg_loss:.2f} | validation avg. loss: {val_avg_loss:.2f}")

Epochs: 0 | epoch avg. loss: 41899.58 | validation avg. loss: 247.35


In [ ]:
model.eval()
predictions = []
real = []

for batch in testloader:
    
    output = model(batch.to(device))
    predictions.append(output.detach().cpu().numpy())
    real.append(get_y(batch).detach().cpu().numpy())

predictions = np.concatenate(predictions)
real = np.concatenate(real)

In [ ]:
model.eval()

In [ ]:
predictions.shape

In [ ]:
real.shape

In [ ]:
batch = next(iter(testloader))

In [ ]:
batch.to(device)
output = model(batch)
criterion(output, get_y(batch))

In [ ]:
plt.scatter(output.detach().cpu().numpy(), get_y(batch).detach().cpu().numpy())
plt.axline((0.7,0.7), slope=1)

In [ ]:


plt.scatter(real, predictions, alpha=0.1)
plt.axline((0.7,0.7), slope=1)

plt.ylabel('Predicted diversity')
plt.xlabel('Observed diversity')

In [ ]:
real.shape

In [ ]:
predictions.shape

In [ ]:
import matplotlib.pyplot as plt


plt.scatter(batch.x[:,0], output.detach().cpu().numpy())
plt.ylabel('Predicted node time')
plt.xlabel('Observed node time')